In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio

arr = sio.loadmat('data_person_trim.mat')
train_trim = arr['train_trim']
test_trim = arr['test_trim']

Ntrn = len(train_trim)       #Ntrn=size(train_trim,1)  # 학습데이터의 수
Ntst = len(test_trim)        #Ntst=size(test_trim,1)   # 테스트데이터의 수
Isize = test_trim.shape[1]   #Isize=size(train_trim,2) # 입력벡터의 차원
Mcls = 50                   #Mcls = 50                # 클래스의 수

Ntrn, Ntst, Isize, Mcls


(150, 300, 2000, 50)

In [2]:
# PCA에 의한 특징 추출

X=train_trim;                            # 학습데이터 행렬
meanfig= np.mean(X, axis=0) 
M=np.tile(meanfig,(len(X),1))            # 평균벡터를 데이터 수만큼 반복한 행렬
S= np.array(X-M) @ np.transpose(X-M) / Ntrn  # 고유치분석을 위한 행렬을 계산
V,D,U=np.linalg.svd(S)                   # 고유치분석
W=np.dot(np.transpose(X-M), V);          # 고유벡터 찾기

#X=train_trim;                   # 학습데이터 행렬
#meanfig = mean(X);              # 학습데이터 행렬의 평균
#M=repmat(meanfig,size(X,1),1);  # 평균벡터를 데이터 수만큼 반복한 행렬
#S=(X-M)*(X-M)'/Ntrn;            # 고유치분석을 위한 행렬을 계산
#[V,D,U]=svd(S);                 # 고유치분석
#W=(X-M)'*V;                     # 고유벡터 찾기
S.shape, V.shape, W.shape

((150, 150), (150, 150), (2000, 150))

In [3]:
eval=np.diag(D);                   # PCA로 축소할 차원 결정
SM = 0
for i in range(150):
    SM = SM + eval[i,i]

SM1=0
for i in range(150):
    SM1 = SM1 + eval[i,i]
    if SM1/SM > 0.95:
        break
        
SM, SM1, i


(50.92572612721602, 48.439964230192444, 68)

In [4]:


Ydim=i;
Wo, Wi= np.linalg.qr(W[:,0:Ydim]);           # 변환행렬의 직교화
Y=np.transpose(np.dot(Wo.T, X.T));            # PCA에 의한 차원축소 (학습데이터)
Yt=np.transpose(np.dot(Wo.T, test_trim.T));  # PCA에 의한 차원축소 (테스트데이터)

Wo.shape, Wi.shape, Y.shape, Yt.shape


((2000, 68), (68, 68), (150, 68), (300, 68))

In [5]:
# LDA에 의한 특징추출

Sw = np.zeros((Ydim, Ydim))                # Within-Scatter Matrix의 계산
m = np.zeros((Mcls, Ydim))  #????????????  # Within-Scatter Matrix의 계산
for i in range(Mcls):       
    C=Y[i*3:(i+1)*3-1,:] 
    #print(i, C.shape, Y.shape)
    Sw = Sw + 3 * np.cov(C.T) 
    m[i,:]=np.mean(C, axis=0)

#C.shape, Sw.shape
m.shape

(50, 68)

In [6]:

Sb=Mcls * np.cov(m.T);  # Between-Scatter Matrix의 계산
Vf,Df,Uf=np.linalg.svd( np.dot(np.linalg.inv(Sw),Sb) ) # LDA 변환행렬 찾기


Sw.shape, Sb.shape, Vf.shape, Df.shape, Uf.shape

((68, 68), (68, 68), (68, 68), (68,), (68, 68))

In [7]:
Z=np.transpose(np.dot(Vf.T, Y.T))   # LDA에 의한 특징추출 (학습데이터)
Zt=np.transpose(np.dot(Vf.T, Yt.T)) # LDA에 의한 특징추출 (테스트데이터)

Z.shape, Zt.shape

((150, 68), (300, 68))

In [8]:
import scipy.io
#save feature_person W Vf Y Yt Z Zt; # 추출된 특징 저장

#arr = np.array((W, Vf, Y, Yt, Z, Zt))

np.save("feature_person_W",  W )
np.save("feature_person_Vf",  Vf )
np.save("feature_person_Y",  Y )
np.save("feature_person_Yt",  Yt )
np.save("feature_person_Z",  Z )
np.save("feature_person_Zt",  Zt )
